In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 25 13:50:46 2019

@author: chenzhm
"""

'\nCreated on Mon Nov 25 13:50:46 2019\n\n@author: chenzhm\n'

In [3]:
import yaml, time, gc, numba
import tensorflow as tf
import numpy as np

import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import PointCloud2, PointField, Image
import message_filters

from pyquaternion import Quaternion

ModuleNotFoundError: No module named 'numba'

In [6]:
class Frame2PointCloud():
    
    def __init__(_intrinsic, camera_ids, 
                 camera_poses, width=512, height=424):
        self.camera_ids = camera_ids
        self.camera_poses = camera_poses
        self.width = width
        self.height = height
        
        # width, height
        self.batch_size = len(self.camera_ids)
        self.cx_list=[]
        self.cy_list=[]
        self.fx_list=[]
        self.fy_list=[]
        for camera_id in camera_ids:
            self.cx_list.append(_intrinsic[camera_id][2])
            self.cy_list.append(_intrinsic[camera_id][5])
            self.fx_list.append(_intrinsic[camera_id][0])
            self.fy_list.append(_intrinsic[camera_id][4])
            
    @numba.jit
    def project(color, depth, uvzrgb):
        numpoints = len(uvzrgb)
        for i in range(numpoints):
            u,v,z,r,g,b = uvzrgb[i]
            if depth[v, u] < z:
                color[v, u] = (r, g, b)
                depth[v, u] = z
    def get_transform_matrix(self, camera_id):
        pose = self.camera_poses['poses'][camera_id]
        extr_q = Quaternion(pose['rotation']['w'], 
                            pose['rotation']['x'], 
                            pose['rotation']['y'], 
                            pose['rotation']['z'])
        extr_q = extr_q.rotation_matrix
        extr_t = np.array([pose['translation']['x'], 
                           pose['translation']['y'],
                           pose['translation']['z']])
        extr_t = np.reshape(extr_t, (3,1))
        transform_matrix = np.concatenate([extr_q, extr_t], 
                                          axis=1)
        transform_matrix = np.concatenate([transform_matrix,
                                           [[0,0,0,1]]],
                                          axis=0)
        return np.float32(transform_matrix)
    
    def build_registration_graph(self, width, height, 
                                 batch_size, 
                                 cx, cy, fx, fy):
        """
        建立点云注册、融合计算图，将多相机rgb和depth图像转化并统一为世界坐标下的点云
         深度图像注册点云公式：
                        z = d/1000
                        x = (u-cx)*z/fx
                        y = (v-cy)*z/fy
            d = depth[v,u]
            u:像素横坐标，u in（0，width-1）
            v：像素纵坐标，v in（0，height-1）
            cx，cy：像素中心，相机内参，约为width/2, height/2
            fx，fy：焦距，相机内参
            x,y,z:点云在相机坐标系下的坐标

        args:
            width: number. 512
            height: number. 424
            batch_size: number. 生成点云使用的相机数量，即图像对(color,depth)数量
            cx, cy, fx, fy: list for camera intrinsic. length = batch_size。 各相机内参列表
        returns:
            color：rgb图像tensor，[batch_size, height, width, 3]
            depth：depth图像tensor, [batch_size, height, width, 1]
            xyzrgb：世界坐标系下的多相机融合点云tensor, [-1, 6]

        """
        self.graph = tf.Graph()
        
        with self.graph.as_default():
            color = tf.placeholder(tf.float32, 
                                   [batch_size, height, width, 3],
                                   name='color_tensor')
            depth = tf.placeholder(tf.float32, 
                                   [batch_size, height, width, 1],
                                   name='depth_tensor')
            u, v = tf.meshgrid(tf.range(width), tf.range(height))
            u, v = tf.to_float(u), tf.to_float(v)
            u = tf.reshape(u, [1, height, width, 1])
            u = tf.tile(u, [batch_size, 1, 1, 1])   #[batch_size, height, width, 1]
            v = tf.reshape(v, [1, height, width, 1])
            v = tf.tile(v, [batch_size, 1, 1, 1])   #[batch_size, height, width, 1]
            cx = tf.constant(cx, tf.float32, [batch_size, 1, 1, 1])
            cy = tf.constant(cy, tf.float32, [batch_size, 1, 1, 1])
            fx = tf.constant(fx, tf.float32, [batch_size, 1, 1, 1])
            fy = tf.constant(fy, tf.float32, [batch_size, 1, 1, 1])

            #注册点云
            z = depth/1000.
            z_fault = tf.ones_like(z)*999
            z = tf.where(tf.equal(z, 0.), z_fault, z)
            z = tf.where(tf.equal(tf.reduce_sum(color, axis=3, 
                                                keep_dims=True), 0.), z_fault, z)
            x = (u-cx)*z/fx
            y = (v-cy)*z/fy
            w = tf.ones_like(x, tf.float32)
            color1 = color/255.
            xyzwrgb = tf.concat([x,y,z,w,color1], axis=-1)#[batch_size, height, width, 7]
            xyzwrgb = tf.reshape(xyzwrgb, (batch_size,-1,7))

            #转换点云至世界坐标系，transform_matrix为坐标转换矩阵
            xyzw_list=[]
            for i in range(batch_size):
                transform_matrix = self.get_transform_matrix(self.camera_ids[i])
                transform_matrix = tf.constant(transform_matrix, tf.float32)
                xyzw0 = tf.transpose(xyzwrgb[i,:,:4], (1,0))#[4, width*height]
                xyzw0 = tf.matmul(transform_matrix, xyzw0)
                xyzw0 = tf.transpose(xyzw0, (1,0))
                xyzw_list.append(xyzw0)
            xyzw = tf.stack(xyzw_list, axis=0) #[batch_siz, width*height, 4]
            xyzrgb = tf.concat([xyzw[...,:3], xyzwrgb[...,-3:]], axis=-1)
            xyzrgb = tf.reshape(xyzrgb, (-1, 6))

            #剔除区域外部分点云
            condx0 = tf.greater(xyzrgb[:,0], image_xmin)
            condx1 = tf.less(xyzrgb[:,0], image_xmax)
            condy0 = tf.greater(xyzrgb[:,1], image_ymin)
            condy1 = tf.less(xyzrgb[:,1], image_ymax)
            condz0 = tf.greater(xyzrgb[:,2], image_zmin)
            condz1 = tf.less(xyzrgb[:,2], image_zmax)

            cond = tf.stack([condx0,condx1,condy0,condy1,condz0,condz1], axis=1)
            cond = tf.reduce_all(cond, axis=1)
            indices = tf.where(cond)
            xyzrgb = tf.gather(xyzrgb, indices, axis=0)
            xyzrgb = tf.squeeze(xyzrgb)#[numpoints, 6]

            #顶部视角预处理数据
            indu = tf.to_int32((xyzrgb[:,0:1] - image_xmin)*100)
            indv = tf.to_int32((xyzrgb[:,1:2] - image_ymin)*100)
            intz = tf.to_int32(xyzrgb[:,2:3]*1000)
            intrgb = tf.to_int32(xyzrgb[:,-3:]*255)
            uvzrgb = tf.concat([indu,indv,intz,intrgb],axis=1)
            xyzrgb = tf.identity(xyzrgb, name='xyzrgb')
            uvzrgb = tf.identity(uvzrgb, name='uvzrgb')
        
    def point2msg(pointArray):
        fields = [PointField('x', 0, PointField.FLOAT32, 1),
                  PointField('y', 4, PointField.FLOAT32, 1),
                  PointField('z', 8, PointField.FLOAT32, 1),
                  PointField('b', 12, PointField.FLOAT32, 1),
                  PointField('g', 16, PointField.FLOAT32, 1),
                  PointField('r', 20, PointField.FLOAT32, 1),
                  ]
        objPoints = PointCloud2()
        objPoints.fields = fields
        objPoints.header.frame_id = 'world'
        objPoints.data = pointArray.tostring()
        objPoints.point_step = 24
        objPoints.width = 1
        objPoints.row_step = 24
        objPoints.height = pointArray.shape[0]
        objPoints.is_dense = False
        return objPoints
    
    def make_point_msg():
        fields = [PointField('x', 0, PointField.FLOAT32, 1),
                  PointField('y', 4, PointField.FLOAT32, 1),
                  PointField('z', 8, PointField.FLOAT32, 1),
                  PointField('b', 12, PointField.FLOAT32, 1),
                  PointField('g', 16, PointField.FLOAT32, 1),
                  PointField('r', 20, PointField.FLOAT32, 1),
                  ]
        objPoints = PointCloud2()
        objPoints.fields = fields
        objPoints.header.frame_id = 'world'
        objPoints.point_step = 24
        objPoints.width = 1
        objPoints.row_step = 24
        objPoints.is_dense = False
        return objPoints
    
    def callback(self, *msgs):
        t0 = time.time()
        color_list = []
        depth_list = []
        stamp_list = []
        for msg in msgs:
            _stamp = float('%d.%d' % (msg.header.stamp.secs,
                                      msg.header.stamp.nsecs))
            stamp_list.append(_stamp)
            frame = self._cv_bridge.imgmsg_to_cv2(msg, 
                                    desired_encoding=color_msg.encoding)
            l = len(frame.shape)
            if l == 2: # depth-->[h,w]
                color_list.append(frame)
            if l == 3: # rgb-->[h,w,3]
                depth_list.append(frame)
        stamp = str(np.mean(stamp_list)).split('.')
        
        color = np.stack(color_list, axis=0).reshape(self.batch_size, 
                                                     self.height,
                                                     self.width, 3)
        depth = np.stack(depth_list, axis=0).reshape(self.batch_size, 
                                                     self.height,
                                                     self.width, 3)
        t1 = time.time()
        color_feed = np.float32(color)
        depth_feed = np.float32(depth)
        feed_dict = {self.color_tensor: color_feed, 
                     self.depth_tensor: depth_feed}
        xyzrgb, uvzrgb = self.sess.run([xyzrgb_tensor,uvzrgb_tensor], 
                                  feed_dict=feed_dict)
        t2 = time.time()
        project_color0 = np.ones([300,450,3], np.uint8)*255
        project_depth0 = np.ones([300,450], np.uint16)*80
        self.project(project_color0, project_depth0, uvzrgb)
        msg_proj_color = self._cv_bridge.cv2_to_imgmsg(cvim=project_color0,
                                                       encoding='8UC3')
        msg_proj_depth = self._cv_bridge.cv2_to_imgmsg(cvim=project_depth0,
                                                       encoding='16UC1')
        msg_proj_color.header.stamp.secs = int(stamp[0])
        msg_proj_color.header.stamp.nsecs = int(stamp[1])
        msg_proj_depth.header.stamp.secs = int(stamp[0])
        msg_proj_depth.header.stamp.nsecs = int(stamp[1])
        self.proj_color_pub.publish(msg_proj_color)
        self.proj_depth_pub.publish(msg_proj_depth)
        t3 = time.time()
        
        # Point Cloud msg
        point_msg = self.make_point_msg()
        point_msg.header.stamp.secs = int(stamp[0])
        point_msg.header.stamp.nsecs = int(stamp[1])
        point_msg.height = xyzrgb.shape[0]
        point_msg.data = xyzrgb.tostring()
        self.points_pub.publish(point_msg)
        t4 = time.time()
        
    def run(self):
    
        config = tf.ConfigProto(allow_soft_placement=True, 
                                inter_op_parallelism_threads=0, 
                                intra_op_parallelism_threads=0)
        config.gpu_options.per_process_gpu_memory_fraction=0.5
        self.sess = tf.Session(config=config)
        self.build_registration_graph(self.width, 
                                      self.height, 
                                      self.batch_size,
                                      self.cx_list, 
                                      self.cy_list,
                                      self.fx_list, 
                                      self.fy_list)
        self.color_tensor = self.graph.get_tensor_by_name('color_tensor:0')
        self.depth_tensor = self.graph.get_tensor_by_name('depth_tensor:0')
        self.xyzrgb_tensor = self.graph.get_tensor_by_name('xyzrgb:0')
        self.uvzrgb_tensor = self.graph.get_tensor_by_name('uvzrgb:0')
        
        rospy.init_node('x', anonymous=True)
        self.points_pub = rospy.Publisher('/world/merge_pointcloud',
                                          PointCloud2, 
                                          queue_size=1)
        self.proj_color_pub = rospy.Publisher('/world/proj_color', 
                                              Image, 
                                              queue_size=5)
        self.proj_depth_pub = rospy.Publisher('/world/proj_depth', 
                                              Image, 
                                              queue_size=5)
        
        self._cv_bridge = CvBridge()
        sub_list = []
        for camera_id in self.camera_ids:
            color_sub = message_filters.Subscriber('/%s/sd/image_color_rect'
                                                   % camera_id, Image)
            depth_sub = message_filters.Subscriber('/%s/sd/image_depth_rect'
                                                   % camera_id, Image)
            sub_list.append(color_sub)
            sub_list.append(depth_sub)
            
        ts = message_filters.ApproximateTimeSynchronizer(sub_list, 
                                                         15,
                                                         0.05)
        tf.regiterCallback(self.callback)
        rospy.loginfo('[Info]: <<<<<<<<<<--Node start-->>>>>>>>>>')
    

NameError: name 'numba' is not defined

In [ ]:
if __name__ == '__main__':
    
    _intrinsic = yaml.load(open('../cfg/camera_intrinsic.yaml'))
    _camera_pose = yaml.load(open('../cfg/camera_poses.yaml'))
    camera_id_list=['k01', 'k10','k03','k04']
    
    frame2point = Frame2PointCloud(_intrinsic=_intrinsic,
                                   camera_ids=camera_id_list,
                                   camera_poses=_camera_pose,
                                   width=512, 
                                   height=424)
    frame2point.run()
    try:
        rospy.spin()
    except Exception as e:
        frame2point.sess.close()
        rospy.logerr('[Error]: Frame2PointCloud Exceptions--> %s' % e)
        